This program aims to find local influencers in an undirected social network,
regardless human or animal.
The program is composed of two modules:

1. Graph clustering.

Splits the social network into several dense subgraphs, which
represent a community.
The strategies used are Markov clustering and spectral clustering.

2. Finding the most influental user in a community

The most influental user is defined by the user with the highest centrality.
The measures of centrality used are eigenvector centrality, 
degree centrality and closeness centrality. 

Therefore, this project is a survey of
some density based clustering methods
and centrality measures.

Importing necessary modules:

In [1]:
import dgl
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.preprocessing
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

Loading the datasets.

In [2]:
data_source = [
    r'.\datasets\karate\karate-club.csv',
    r'.\datasets\soc-dolphins\soc-dolphins.csv',
    r'.\datasets\aves-wildbird-network-3\aves-wildbird-network-3.csv',
    r'.\datasets\aves-wildbird-network\aves-wildbird-network.csv',
    r'.\datasets\ca-netscience\ca-netscience.csv',
    r'.\datasets\soc-wiki-Vote\soc-wiki-Vote.csv',
]

nx_graph = []

for i in range(len(data_source)):
    dat = pd.read_csv(data_source[i])
    n1 = dat['node_1'].to_numpy()
    n2 = dat['node_2'].to_numpy()

    gra = dgl.graph((n1, n2))
    nx_graph.append(gra.to_networkx().to_undirected())

First, the Markov clustering method will be performed on the graph.
This uses a random walk technique, which is itself a Markov chain, to identify clusters.

It is observed that for an undirected and unweighted graph, nodes of higher degree are less likely 
to have edges with nodes in another cluster.

First, we construct the adjacency matrix:

In [3]:
matrix = nx.to_scipy_sparse_array(nx_graph[2])
for i in range(0, 6):
    print(nx_graph[i])

MultiGraph with 35 nodes and 78 edges
MultiGraph with 63 nodes and 159 edges
MultiGraph with 170 nodes and 1615 edges
MultiGraph with 203 nodes and 11780 edges
MultiGraph with 380 nodes and 914 edges
MultiGraph with 890 nodes and 2914 edges


A Markov clustering algorithm takes in a normalized matrix, an inflation factor and an expansion factor.

The expansion factor is the length of a random walk, while the inflation factor is
directly correlated to the number of clusters.

It directly returns the clusters.

The following code uses the module from
https://github.com/GuyAllard/markov_clustering

In [4]:
import markov_clustering as mc

# Calculate the quality score of clusters, determined by expansion and inflation factors.
def mc_loop(mat, exp, infl):
    res = mc.run_mcl(mat, expansion=exp, inflation=infl)
    clu = mc.get_clusters(res)

    mod = mc.modularity(res, clu)
    return mod

# Return the best set of clusters.
def mc_loop_final(mat, exp, infl):
    res = mc.run_mcl(mat, expansion=exp, inflation=infl)
    return mc.get_clusters(res)


In [5]:
# Train the Markov clustering method for the expansion factor, by increasing inflation factor.
def mc_train(mat, expand, start, end):
    max_infl = max_mod = 0

    mod_series = []

    for inflation in [i / 10 for i in range(start, end)]:
        Q = mc_loop(mat, expand, inflation)
        mod_series.append(Q)

        if (Q > max_mod):
            max_mod = Q
            max_infl = inflation
        print(f"For expansion = {expand} inflation = {inflation}, modularity = {Q}")

    return max_infl, max_mod, mod_series

# Draw the graph of modularity over inflation factors.
def draw_graph(inflat_ser, mod_ser_exp, exp_start, exp_end):
    i = 0
    for c in range(exp_start, exp_end+1):
        # print(c, i)
        plt.plot(inflat_ser, mod_ser_exp[i], label = f'expansion factor = {c}', linestyle = "--")
        i += 1
    plt.xlabel('inflation factor')
    plt.ylabel('modularity')
    plt.legend()
    plt.show()

# The entire Markov clustering model.
def markov_grand_loop(matr, expand_start, expand_end, infl_start, infl_end):

    max_expand = grand_max_infl = grand_max_modularity = 0

    inflation_series = range(infl_start, infl_end)
    inflation_series = [inf / 10 for inf in inflation_series]

    mod_ser_by_expansion = []

    for i in range(expand_start, expand_end+1):

        curr_max_infl, curr_max_mod, mod_ser = mc_train(matr, i, infl_start, infl_end)

        mod_ser_by_expansion.append(mod_ser)

        if grand_max_modularity < curr_max_mod:
            max_expand = i
            grand_max_infl = curr_max_infl
            grand_max_modularity = curr_max_mod
        
        # print(max_expand, grand_max_infl,  grand_max_modularity)
    
    draw_graph(inflation_series, mod_ser_by_expansion, expand_start, expand_end)

    final_cluster = mc_loop_final(matr, max_expand, grand_max_infl)
    return final_cluster

# Return the list of clusters.
def final_clean(clust):
    cl = []
    for i in range(len(clust)):
        lis = list(clust[i])
        cl.append(lis)
    return cl

Next, we explore another method which is the spectral clustering.
It uses the graph Laplacian and some of its eigenvectors to identify
dense clusters.

To find these clusters, we need to use a classical clustering method,
such as the k-means clustering.

First we construct a graph Laplacian:

In [6]:
def laplacian(G):
    return nx.laplacian_matrix(G).toarray()

Now we perform spectral clustering:

In [7]:
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
import statistics
import math

#The spectral clustering method using a k-means method:
def spectral(G, n = 3):
    lap = laplacian(G)
    eigenvals, eigenvecs = np.linalg.eig(lap)

    eigenvecs = eigenvecs[:,np.argsort(eigenvals)]
    eigenvals = eigenvals[np.argsort(eigenvals)]

    km = KMeans(n)
    km.fit(eigenvecs[:,1:n])
    labels = km.labels_.tolist()

    node_no = list(range(1, G.number_of_nodes() + 1))
    node_no = np.array(node_no).tolist()

    node_and_label = dict(zip(node_no, labels))

    return node_and_label

#With K-medoid method 
def spectral_2(G, n = 3):
    lap = laplacian(G)
    eigenvals, eigenvecs = np.linalg.eig(lap)

    eigenvecs = eigenvecs[:,np.argsort(eigenvals)]
    eigenvals = eigenvals[np.argsort(eigenvals)]

    km = KMedoids(n_clusters=n).fit(eigenvecs[:,1:n])
    labels = km.labels_.tolist()

    node_no = list(range(1, G.number_of_nodes() + 1))
    node_no = np.array(node_no).tolist()

    node_and_label = dict(zip(node_no, labels))

    return node_and_label

'''
def spec_train(G, start, end):
    for x in range(start, end+1):
        lab = spectral(G, n = x)
'''
# Return the list of clusters from spectral clustering.    
def spectral_post_process(n, node_and_labl):
    clusters = []
    for i in range(n):
        clusters.append([])
    for j in node_and_labl:
        clusters[node_and_labl.get(j)].append(j)
    return clusters

#Calculate the skewedness of clusters. The higher the number,
# the more unbalanced the clusters.
def balance_score_clusters(clust):
    num_of_node_in_cluster = []
    for a in clust:
        num_of_node_in_cluster.append(len(a))
    return statistics.stdev(num_of_node_in_cluster)

#The entire spectral clustering model.
def spectral_grand_loop(graph, k_start, k_end):

    k_m = []
    dev_soc = []

    min_std_dev = 99999

    for r in range(k_start, k_end+1):

        k_m.append(r)

        print(r)

        pre_cluster = spectral_2(graph, r)
        fin_cluster = spectral_post_process(r, pre_cluster)
        dev_score = balance_score_clusters(fin_cluster)

        if dev_score < min_std_dev:
            min_std_dev = dev_score

        dev_soc.append(dev_score)
        # print(f"For k = {r}, the standard deviation for clusters is {dev_score}")
    
    plt.plot(k_m, dev_soc, linestyle="--")
    plt.xlabel("k as in k-means/k-medoid clustering")
    plt.ylabel("standard deviation of clustering labels")
    plt.show()
    
    best_k = k_m[ dev_soc.index(min_std_dev) ]

    best_pre_cluster = spectral(graph, best_k)
    return spectral_post_process(best_k, best_pre_cluster)



Now we test the Louvain community detection algorithm:

In [13]:
def louvain_algo(G):
    return nx.algorithms.community.louvain_communities(G, resolution = 0.98)

Now we place the nodes into clusters:

In [8]:
def separate_communities(Dgraph, communities):
    comm = []
    for i in communities:
        if len(i) <= 4:
            continue
        else:
            # print(i)
            co = Dgraph.subgraph(i)
            comm.append(co)
    return comm

With the disjoint clusters ready,
we can calculate the centrality of each node in each subgraph.
The measures of centrality used are eigencentrality, degree centrality and
closeness centrality

In [9]:
#Close centrality of a graph
def close_centrality(G, lst):
    centr = nx.closeness_centrality(G)
    sorted_centr = sorted(centr.items(), key = lambda centr: centr[1])

    sorted_centr.reverse()
    # print(sorted_centr)

    lst.append(sorted_centr[0][0])

#Eigencentrality of a graph
def eigencentrality(G, lst):
    g2 = nx.DiGraph(G)
    ctr = nx.eigenvector_centrality(g2)

    sorted_ctr = sorted(ctr.items(), key = lambda ctr: ctr[1])
    sorted_ctr.reverse()
    # print(sorted_ctr)

    lst.append(sorted_ctr[0][0])

#Degree centrality of a graph
def degree_centrality(G, lst):
    ctr = nx.degree_centrality(G)

    sorted_ctr = sorted(ctr.items(), key = lambda ctr: ctr[1])
    sorted_ctr.reverse()
    # print(sorted_ctr)

    lst.append(sorted_ctr[0][0])

#Identify list of influencers in each community in a graph
def create_list_of_local_influencers(communi, mode):
    influencer_list = []
    for com in communi:
        if mode == 1:
            degree_centrality(com, influencer_list)
        if mode == 2:
            close_centrality(com, influencer_list)
        if mode == 3:
            eigencentrality(com, influencer_list)
    return influencer_list

In [19]:
def markov_test():
    i = 6
    matrix = nx.to_scipy_sparse_array(nx_graph[i-1])
    c = markov_grand_loop(matrix, 2, 6, 11, 40)
    final = final_clean(c)
    communidad = separate_communities(nx_graph[i-1], final)
    li = create_list_of_local_influencers(communidad, 3)
    print(f"List of influencers in dataset #{i}: {li}")

def spectral_test():
    i = 4
    final = spectral_grand_loop(nx_graph[i-1], 2, 20)
    communi = separate_communities(nx_graph[i-1], final)
    lis = create_list_of_local_influencers(communi, 3)
    print(f"List of influencers in dataset {i}: {lis}")

def louvain_test():
    i = 6
    final = louvain_algo(nx_graph[i-1])
    communi = separate_communities(nx_graph[i-1], final)
    lis = create_list_of_local_influencers(communi, 1)
    print(f"List of influencers in dataset {i}: {lis}")

louvain_test()



List of influencers in dataset 6: [8, 123, 389, 542, 170, 273, 550, 657, 762, 376]
